## WordVector를 이용한 네이버 영화평 감정 분석
- 194~218

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Twitter
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_train.txt'
test_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_test.txt'

In [ ]:
def read_data(filename):
  with open(filename, 'r', encoding = 'utf-8') as f:
    # 한문장씩 끊어서 탭 기준으로 분리해서 리스트에 추가
    result = [line.split('\t') for line in f.read().splitlines()]

    # 칸 이름 뺴고 저장
    result = result[1:]
  return result

In [ ]:
train_tmp = read_data(train_path)
test_tmp = read_data(test_path)

In [ ]:
train_tmp[0:10]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1'],
 ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'],
 ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'],
 ['9443947',
  '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'],
 ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'],
 ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]

In [ ]:
# 함수화
twitter = Twitter()

def getNounAdjective(text):
  # 단어 40개 추가할 리스트
  stems = [""] * 40

  # 인덱스 설정 , 40개 리스트의 마지막 인덱스는 39
  stems_last_index = (len(stems) - 1)

  stems_index = 0
  
  # 문장 품사 태깅
  tagged = twitter.pos(text, stem = True)

  for i in range(0, len(tagged)):
    # 명사나 형용사면 단어 추가
    if (tagged[i][1] == 'Noun') or (tagged[i][1] == 'Adjective'):
      stems[stems_index] = tagged[i][0]

      # 단어 40개 다 추가했으면 멈춤
      if stems_index == stems_last_index:
        break

      stems_index +=1

  # 명사나 형용사가 1개면 리턴 안받기
  if stems_index > 2:
    return stems

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
def kor_movie():

  # 학습 데이터로 사용할 리뷰데이터 저장 리스트
  train_x = []

  # 정답 데이터로 사용할 0,1 을 저장 리스트
  train_y = []
  for i in tqdm(range(len(train_tmp))):
    # getNounAdjective 함수 써서 명사랑 형용사만 리턴
    noun_adjective = getNounAdjective(train_tmp[i][1])

    # None값이 아니면
    if noun_adjective is not None:

      # 리스트에 추가
      train_x.append(noun_adjective)

      # 0,1 ,값을 정수로 변환해서 저장
      train_y.append(int(train_tmp[i][2]))

  # 검증 데이터로 사용할 리뷰데이터 저장 리스트
  test_x = []

  # 검증 데이터의 정답 리스트
  test_y = []
  for i in tqdm(range(len(test_tmp))):
    # getNounAdjective 함수 써서 명사랑 형용사만 리턴
    noun_adjective = getNounAdjective(test_tmp[i][1])

    # None값이 아니면
    if noun_adjective is not None:

      # 리스트에 추가
      test_x.append(noun_adjective)

      # 0,1 ,값을 정수로 변환해서 저장
      test_y.append(int(train_tmp[i][2]))
  return (train_x, train_y), (test_x, test_y)

In [ ]:
(X_train, y_train) , (X_test, y_test) = kor_movie()

  0%|          | 0/150000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
del train_tmp
del test_tmp

In [ ]:
# X_train 단어를 워드 벡터로 변환
# size = 20 , 20 개 칸수
word2vector = Word2Vec(X_train, size =20)

# 워드벡터 계산할때 사용한 변수들을 메모리에서삭제
word2vector.init_sims(replace = True)

In [ ]:
# word2vector에 저장된 단어들을 딕셔너리 객체로 리턴후 키값 리턴
word_vector_keys =  word2vector.wv.vocab.keys()
word_vector_keys

dict_keys(['더빙', '진짜', '짜증나다', '목소리', '', '흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '가볍다', '무재', '다그', '래서', '추천', '교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정', '의', '익살스럽다', '스파이더맨', '이쁘다', '막', '걸음', '마', '세', '초등학교', '학년', '생인', '반개', '아깝다', '움', '원작', '긴장감', '제대로', '별', '욕', '이응경', '생활', '년', '정말', '발', '해도', '그것', '납치', '감금', '반복', '드라마', '가족', '사람', '네', '액션', '있다', '몇', '안되다', '왜케', '낮다', '꽤', '볼', '데', '헐리우드', '화려하다', '인피니트', '짱', '볼때', '향수', '자극', '허진호', '감성', '절제', '멜로', '달인', '울면', '손', '때', '뻔', '이범수', '드럽다', '담백하다', '깔끔하다', '좋다', '로만', '자꾸', '그', '것', '취향', '존중', '다지', '내생', '극장', '가장', '노잼', '노', '감동', '임', '스토리', '어거지', '냥', '매번', '긴장', '재밋음', '바스코', '이기', '락스', '코', '바비', '아이돌', '그냥', '안달', '굿바이', '레닌', '표절', '이해', '왜', '뒤', '갈수록', '이건', '깨알', '캐스팅', '산뜻하다', '용구성', '일드', '위', '변명', '놈', '착하다', '절대', '아니다', '걸', '나름', '심오하다', '뜻', '듯', '학생', '선생', '재미없다', '지루하다', '같다', '음식', '도', '만찬', '별로', '핀란드', '풍경', '평범하다', '수작', '는걸', '주제', '중반', '납득', '수', '그렇다', '꼭', '

In [ ]:
# X_train 문자열을 word2vector로 변환해서 저장할 배열
X_train_vector = []

for index in tqdm(range(len(X_train))):
  input = []

  for word in X_train[index]:
    # X_train의 단어가 word_vector_keys에 있으면 추가
    # 없는 경우는 빈도수가 낮은 경우 없음
    if word in word_vector_keys:
      input.append(word2vector[word].tolist())
    else:
      input.append(word2vector[""].tolist())
  X_train_vector.append(input)

# 타입 변환
X_train_vector = np.array(X_train_vector, dtype = np.float32)

# 삭제
del X_train

  0%|          | 0/125008 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [ ]:
# X_test 문자열을 word2vector로 변환해서 저장할 배열
X_test_vector = []

for index in tqdm(range(len(X_test))):
  input = []

  for word in X_test[index]:

    if word in word_vector_keys:
      input.append(word2vector[word].tolist())
    else:
      input.append(word2vector[""].tolist())
  X_test_vector.append(input)

# 타입 변환
X_test_vector = np.array(X_test_vector, dtype = np.float32)

del X_test

  0%|          | 0/41727 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [ ]:
# 모델 
model = Sequential()

# input_shape (단어 갯수(40단어) , 단어 당 칸수(20칸, 단어 벡터))
model.add(LSTM(units = 128, input_shape=(40, 20)))

#
model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               76288     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 76,417
Trainable params: 76,417
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_train = np.array(y_train, dtype = np.float32)
y_test = np.array(y_test, dtype = np.float32)

In [ ]:
model.compile(
               loss="binary_crossentropy",
               optimizer=Adam(lr=1e-3),
               metrics=["acc"]
              )
model.fit(X_train_vector, y_train, batch_size =1000, epochs = 100 )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
126/126 [==============================] - 12s 34ms/step - loss: 0.5217 - acc: 0.7259
Epoch 2/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4517 - acc: 0.7908
Epoch 3/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4434 - acc: 0.7955
Epoch 4/100
126/126 [==============================] - 5s 36ms/step - loss: 0.4389 - acc: 0.7965
Epoch 5/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4381 - acc: 0.7971
Epoch 6/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4315 - acc: 0.7993
Epoch 7/100
126/126 [==============================] - 4s 34ms/step - loss: 0.4291 - acc: 0.8008
Epoch 8/100
126/126 [==============================] - 4s 34ms/step - loss: 0.4282 - acc: 0.8013
Epoch 9/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4209 - acc: 0.8064
Epoch 10/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4227 - acc: 0.8037
Epoch 11/100
126/126 [======

In [ ]:
model.evaluate(X_test_vector, y_test)

1304/1304 [==============================] - 8s 6ms/step - loss: 1.3810 - acc: 0.4998


[1.3810056447982788, 0.4997963011264801]